In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.python.platform import gfile
from imageio import imread
from skimage.transform import resize
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display, HTML

# This module defines the show_graph() function to visualize a TensorFlow graph within Jupyter.

# As far as I can tell, this code was originally written by Alex Mordvintsev at:
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb

# The original code only worked on Chrome (because of the use of <link rel="import"...>, but the version below
# uses Polyfill (copied from this StackOverflow answer: https://stackoverflow.com/a/41463991/38626)
# so that it can work on other browsers as well.

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

def ColorMask(Mask, Color):
    RGBImage = np.stack([Mask, Mask, Mask], axis = 2)
    return np.multiply(RGBImage, Color)

def getProbIndex(ProbMatrix):
    MaxIndex = np.argmax(np.squeeze(ProbMatrix), axis = 2)
    return MaxIndex

def getObjective(IndexMatrix, Objective):
    return (IndexMatrix == Objective).astype('float32')

In [ ]:
sess = tf.InteractiveSession()
f = gfile.FastGFile("/media/anubis/Data/OpenVINO/UnetFull/UNetFull.pb", 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
f.close()

In [ ]:
sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
tf.import_graph_def(graph_def)
show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
output_tensor = sess.graph.get_tensor_by_name('import/conv2d_23/truediv:0')
input_tensor = sess.graph.get_tensor_by_name('import/input_1:0')
model_image_size = (input_tensor.shape.dims[1].value, input_tensor.shape.dims[2].value)

In [ ]:
I = imread('/media/anubis/Data/OpenVINO/Datasets/PH2/validation/images/IMD256.bmp')
input_image_size = (I.shape[0], I.shape[1])
image = resize(I, model_image_size, mode='constant', anti_aliasing=False)
image = np.expand_dims(image, axis = 0)
prediction = sess.run(output_tensor, {input_tensor: image})

PrdIndex = getProbIndex(prediction)
PrdMask = getObjective(PrdIndex, 0)
PrdMask = ColorMask(PrdMask, [1, 0.25, 1])
PrdMask = resize(PrdMask, input_image_size, mode='constant', anti_aliasing=False)

plt.imshow(I, interpolation='none')
plt.imshow(PrdMask, interpolation='none', alpha=0.4)
plt.tick_params(axis='both',which='both',bottom=False,left=False,labelbottom=False,labelleft=False)
plt.title('IMD256' + ' Predicted')
plt.show()